In [1]:
from pandarallel import pandarallel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gc
import pickle5 as pickle

In [3]:
ls

data/                 part1.ipynb          prod2vec.pdf  raw_data_proc.ipynb
get_categories.ipynb  part2.ipynb          __pycache__/  utils.py
get_metrics.ipynb     part3.ipynb          pys/          w2vec/
LearningToRank.ipynb  prob_features.ipynb  raw_data/


In [2]:

df = pd.read_pickle("data/df_features.pkl")

with open('data/num_of_to_cart.pkl', 'rb') as handle:
     num_of_carts = pickle.load(handle)

with open('data/num_of_view.pkl', 'rb') as handle:
    num_of_views = pickle.load(handle)

#with open('data/num_of_order.pkl', 'rb') as handle:
#   num_of_orders = pickle.load(handle)

num_of_views = dict((str(k), v) for k, v in num_of_views.items())
num_of_carts = dict((str(k), v) for k, v in num_of_carts.items())

categories = pd.read_pickle("data/categories.pkl")
categories.head()
categories['itemid'] = categories['itemid'].astype(int)

categories_1 = dict(categories[['itemid','category_name_1_level_en']].values.tolist())
categories_2 = dict(categories[['itemid','category_name_2_level_en']].values.tolist())

del categories
gc.collect()

def get_prediction_and_score(x):
    if not isinstance(x, np.int64):
        #print(x)
        prediction = x[0]
        score = x[1]
    else:
        prediction = x
        score = 0.0
    return prediction, score

def transform(df):
    exploded = df.explode('ovr_pred')
    exploded = exploded.reset_index()
    #exploded = exploded.drop(['index'], axis=1)
    #exploded.drop(['to_cart','view'], axis=1, inplace=True)
    pred_score = exploded['ovr_pred'].apply(get_prediction_and_score)
    
    exploded['prediction'] = [x[0] for x in pred_score]
    exploded['score'] = [x[1] for x in pred_score]
    exploded['target'] = exploded[['order','prediction']].apply(lambda x: 1 if x[1] in x[0] else 0, axis=1)
    exploded['mean_score_for_prediction'] = exploded.groupby(['prediction'])['score'].transform('mean')
    
    exploded['num_of_cart'] = exploded['prediction'].map(num_of_carts)
    exploded['num_of_views'] = exploded['prediction'].map(num_of_views)
    #exploded['num_of_ords'] = exploded['prediction'].map(num_of_orders)
    exploded['num_of_cart'].fillna(0, inplace=True)
    exploded['num_of_views'].fillna(0, inplace=True)
    #exploded['num_of_ords'].fillna(0, inplace=True)
    
    exploded['category_1'] = exploded['prediction'].map(categories_1)
    exploded['category_2'] = exploded['prediction'].map(categories_2)
    exploded['category_1'].fillna("None", inplace=True)
    exploded['category_2'].fillna("None", inplace=True)
    

    exploded['mean_ovr'].fillna(0, inplace=True)
    exploded['avg_w2vec'].fillna(0, inplace=True)
    exploded['avg_cosine'].fillna(0, inplace=True)
    
    #Были ли предикты для этой сесии от косинусной модели или от в2век
    exploded['is_popular'] = exploded['len_of_pred'] > 0
    
    exploded.drop(['ovr_pred','cos_pred','w2vec_pred','top_pred','order'], axis=1, inplace=True)
    #exploded.fillna(0, inplace=True)
    return exploded


size = df.shape[0]
exp = transform(df)
exp.to_pickle("data/exploded_df.pkl")
del df, num_of_views, num_of_carts, categories_1, categories_2
gc.collect()
exp.head()

,index,view,cart,avg_w2vec,avg_cosine,sum_w2vec,sum_cosine,len_of_w2vec,len_of_cos,len_of_pred,...,intersection_cosine,prediction,score,target,mean_score_for_prediction,num_of_cart,num_of_views,category_1,category_2,is_popular
0,0,"[54712, 54714, 54719, 54720, 1892, 54723, 5472...","[54710, 54711, 54713, 54715, 7521, 54716, 5471...",0.944428,0.454887,47.22141,22.744328,50,50,100,...,0,699536,0.967801,0,0.923842,0.0,0.0,Produkty pitanija,"Masla, sousy, spetsii",True
1,0,"[54712, 54714, 54719, 54720, 1892, 54723, 5472...","[54710, 54711, 54713, 54715, 7521, 54716, 5471...",0.944428,0.454887,47.22141,22.744328,50,50,100,...,0,1178128,0.965379,0,0.921920,0.0,0.0,Produkty pitanija,"Masla, sousy, spetsii",True
2,0,"[54712, 54714, 54719, 54720, 1892, 54723, 5472...","[54710, 54711, 54713, 54715, 7521, 54716, 5471...",0.944428,0.454887,47.22141,22.744328,50,50,100,...,0,471464,0.963017,0,0.849538,0.0,0.0,Produkty pitanija,"Masla, sousy, spetsii",True
3,0,"[54712, 54714, 54719, 54720, 1892, 54723, 5472...","[54710, 54711, 54713, 54715, 7521, 54716, 5471...",0.944428,0.454887,47.22141,22.744328,50,50,100,...,0,1415188,0.962703,0,0.809834,0.0,0.0,Produkty pitanija,"Masla, sousy, spetsii",True
4,0,"[54712, 54714, 54719, 54720, 1892, 54723, 5472...","[54710, 54711, 54713, 54715, 7521, 54716, 5471...",0.944428,0.454887,47.22141,22.744328,50,50,100,...,0,265828,0.962319,0,0.818355,0.0,0.0,Produkty pitanija,"Masla, sousy, spetsii",True


In [3]:
%whos

Variable                   Type              Data/Info
------------------------------------------------------
exp                        DataFrame                  index           <...>783899 rows x 23 columns]
gc                         module            <module 'gc' (built-in)>
get_prediction_and_score   function          <function get_prediction_<...>_score at 0x7f9636581b80>
handle                     BufferedReader    <_io.BufferedReader name='data/num_of_view.pkl'>
np                         module            <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
pandarallel                type              <class 'pandarallel.pandarallel.pandarallel'>
pd                         module            <module 'pandas' from '/h<...>ages/pandas/__init__.py'>
pickle                     module            <module 'pickle5' from '/<...>ges/pickle5/__init__.py'>
plt                        module            <module 'matplotlib.pyplo<...>es/matplotlib/pyplot.py'>
size                       in

In [5]:
ls data/

categories.pkl    num_of_to_cart.pkl   session_item_cart_add.npz
df_features.pkl   num_of_view.pkl      session_item_view.npz
df_scores.pkl     PAB_cart.npz         session_order.npz
exploded_df.pkl   PAB_view.npz         similarities_cart_add.npz
exp_test.pkl      PB_cart.npy          similarities_view.npz
exp_train.pkl     PB_view.npy          top50.pkl
item_mapping.pkl  processed.json
num_of_order.pkl  reverse_mapping.pkl


In [6]:
import scipy.sparse as sp
#with open('old2new_dict.pkl', 'rb') as handle:
#   old2new_dict = pickle.load(handle)
    
sim_view = sp.load_npz("data/similarities_view.npz")
sim_cart = sp.load_npz("data/similarities_cart_add.npz")

#Proba features martices and vecs
PAB_view = sp.load_npz("data/PAB_view.npz")
PAB_cart = sp.load_npz("data/PAB_cart.npz")

with open("data/PB_view.npy", 'rb') as f:
    PB_view = np.load(f).T
    
with open("data/PB_cart.npy", 'rb') as f:
    PB_cart = np.load(f).T

In [7]:
%%time
pandarallel.initialize(8)
def prob_features(row):
    view = row[0]
    cart = row[1]
    prediction = row[2]
    ovr_view = 0
    ovr_cart = 0
    
    prob_view = [0]
    prob_cart = [0]
    
    for item in view:
        prob_view += PAB_view[prediction, item] / PB_view[item]
        sim_view_ = sim_view[prediction, item]
        ovr_view += sim_view_
    pred_view_sim = ovr_view

    for item in cart:
        prob_cart += PAB_cart[prediction, item] / PB_cart[item]
        sim_cart_ = sim_cart[prediction, item]
        ovr_cart += sim_cart_
    pred_cart_sim = ovr_cart
    return pd.Series([pred_cart_sim, pred_view_sim, prob_cart[0], prob_view[0]])

exp[['pred_cart_sim', 'pred_view_sim', 'prob_cart', 'prob_view']] = \
exp[['view', 'cart', 'prediction']].parallel_apply(prob_features, axis=1)

print(np.corrcoef(exp['prob_cart'], exp['target']))

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
[[1.         0.45144777]
 [0.45144777 1.        ]]
CPU times: user 803 ms, sys: 499 ms, total: 1.3 s
Wall time: 25min 59s


In [8]:
%whos

Variable                   Type              Data/Info
------------------------------------------------------
PAB_cart                   csc_matrix          (92874, 0)	1.0\n  (2, 0<...>\n  (765800, 2044964)	1.0
PAB_view                   csc_matrix          (2001301, 0)	1.0\n  (11<...>0\n  (20494, 2044964)	1.0
PB_cart                    ndarray           2044977x1: 2044977 elems, type `float64`, 16359816 bytes (15.601936340332031 Mb)
PB_view                    ndarray           2044977x1: 2044977 elems, type `float64`, 16359816 bytes (15.601936340332031 Mb)
exp                        DataFrame                  index           <...>783899 rows x 27 columns]
f                          BufferedReader    <_io.BufferedReader name='data/PB_cart.npy'>
gc                         module            <module 'gc' (built-in)>
get_prediction_and_score   function          <function get_prediction_<...>_score at 0x7f9636581b80>
handle                     BufferedReader    <_io.BufferedReader name='dat

In [12]:
del PAB_cart, PAB_view, PB_view, PB_cart 

In [13]:
gc.collect()

2681

In [11]:
# exp.to_csv("tmp.csv")

# exp = pd.read_csv("tmp.csv")

In [16]:
exp.drop(["view", "cart"], axis=1, inplace=True)

In [17]:
size = 400000
indices = np.arange(size)
np.random.shuffle(indices)

train_ind = indices[:int(0.8 * size)]
test_ind = indices[int(0.8 * size):]

exp_train = exp.loc[exp['index'].isin(train_ind)]
exp_test = exp.loc[exp['index'].isin(test_ind)]

In [18]:
print(exp_train.shape, exp_test.shape)

(4633278, 25) (1150621, 25)


In [19]:
exp.shape

(5783899, 25)

In [20]:
#expanding mean
global_mean = exp_train['target'].mean()

cat1_target_mean = exp_train.groupby('category_1')['target'].mean()
cumsum = exp_train.groupby('category_1')['target'].cumsum() - exp_train['target']
cumcnt = exp_train.groupby('category_1').cumcount()
exp_train['cat1_encoded_feature'] = cumsum / cumcnt
exp_train['cat1_encoded_feature'].fillna(global_mean, inplace = True)

exp_test['cat1_encoded_feature'] = exp_test['category_1'].map(cat1_target_mean)
exp_test['cat1_encoded_feature'].fillna(global_mean, inplace = True)


cat2_target_mean = exp_train.groupby('category_2')['target'].mean()
cumsum = exp_train.groupby('category_2')['target'].cumsum() - exp_train['target']
cumcnt = exp_train.groupby('category_2').cumcount()
exp_train['cat2_encoded_feature'] = cumsum / cumcnt
exp_train['cat2_encoded_feature'].fillna(global_mean, inplace = True)
exp_train.drop(['category_2'], axis=1, inplace=True)

exp_test['cat2_encoded_feature'] = exp_test['category_2'].map(cat2_target_mean)
exp_test['cat2_encoded_feature'].fillna(global_mean, inplace = True)
exp_test.drop(['category_2'], axis=1, inplace=True)

<ipython-input-20-24f14164ba3b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_train['cat1_encoded_feature'] = cumsum / cumcnt
/home/huvi/anaconda3/envs/ozon/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-20-24f14164ba3b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [21]:
exp_train.to_pickle("data/exp_train.pkl")
exp_test.to_pickle("data/exp_test.pkl")

In [1]:
# import pandas as pd
# exp_train = pd.read_pickle("data/exp_train.pkl")
# exp_test = pd.read_pickle("data/exp_test.pkl")

In [22]:
exp = pd.concat([exp_train, exp_test])
exp = pd.get_dummies(exp, columns=['category_1'], drop_first=True, dtype=bool)
exp.shape
exp.head()

,index,avg_w2vec,avg_cosine,sum_w2vec,sum_cosine,len_of_w2vec,len_of_cos,len_of_pred,mean_ovr,sum_ovr,...,category_1_OZON Express,"category_1_Odezhda, obuv' i aksessuary",category_1_Produkty pitanija,category_1_Sportivnye tovary,category_1_Stroitel'stvo i remont,category_1_Tovary dlja vzroslyh,category_1_Tovary dlja zhivotnyh,category_1_Tsifrovye tovary,"category_1_Turizm, rybalka, ohota",category_1_Vse dlja igr
0,0,0.944428,0.454887,47.22141,22.744328,50,50,100,0.699657,69.965738,...,False,False,True,False,False,False,False,False,False,False
1,0,0.944428,0.454887,47.22141,22.744328,50,50,100,0.699657,69.965738,...,False,False,True,False,False,False,False,False,False,False
2,0,0.944428,0.454887,47.22141,22.744328,50,50,100,0.699657,69.965738,...,False,False,True,False,False,False,False,False,False,False
3,0,0.944428,0.454887,47.22141,22.744328,50,50,100,0.699657,69.965738,...,False,False,True,False,False,False,False,False,False,False
4,0,0.944428,0.454887,47.22141,22.744328,50,50,100,0.699657,69.965738,...,False,False,True,False,False,False,False,False,False,False


In [19]:
import numpy as np
train_ind = np.unique(exp_train['index'])
test_ind = np.unique(exp_test['index'])

In [23]:
import gc
del exp_train, exp_test
gc.collect()

68

In [24]:
exp_train = exp.loc[exp['index'].isin(train_ind)]
exp_test = exp.loc[exp['index'].isin(test_ind)]

In [25]:
print(exp_train.shape, exp_test.shape)

(4633278, 52) (1150621, 52)


In [26]:
exp_train.head()

,index,avg_w2vec,avg_cosine,sum_w2vec,sum_cosine,len_of_w2vec,len_of_cos,len_of_pred,mean_ovr,sum_ovr,...,category_1_OZON Express,"category_1_Odezhda, obuv' i aksessuary",category_1_Produkty pitanija,category_1_Sportivnye tovary,category_1_Stroitel'stvo i remont,category_1_Tovary dlja vzroslyh,category_1_Tovary dlja zhivotnyh,category_1_Tsifrovye tovary,"category_1_Turizm, rybalka, ohota",category_1_Vse dlja igr
0,0,0.944428,0.454887,47.22141,22.744328,50,50,100,0.699657,69.965738,...,False,False,True,False,False,False,False,False,False,False
1,0,0.944428,0.454887,47.22141,22.744328,50,50,100,0.699657,69.965738,...,False,False,True,False,False,False,False,False,False,False
2,0,0.944428,0.454887,47.22141,22.744328,50,50,100,0.699657,69.965738,...,False,False,True,False,False,False,False,False,False,False
3,0,0.944428,0.454887,47.22141,22.744328,50,50,100,0.699657,69.965738,...,False,False,True,False,False,False,False,False,False,False
4,0,0.944428,0.454887,47.22141,22.744328,50,50,100,0.699657,69.965738,...,False,False,True,False,False,False,False,False,False,False


In [27]:
exp_train.to_pickle("exp_train.pkl")
exp_test.to_pickle("exp_test.pkl")

In [30]:
np.corrcoef(exp_train['target'], exp_train['pred_cart_sim'])

array([[1.        , 0.13155936],
       [0.13155936, 1.        ]])